In [1]:
import meep as mp
from meep import Vector3

# Define simulation region (1D-like for stratified media)
resolution = 100  # Grid resolution
cell_size = mp.Vector3(1, 0, 0)  # 1D along x
pml_layers = [mp.PML(thickness=0.5)]

# Materials (isotropic/anisotropic example)
air = mp.Medium(epsilon=1.0)
glass = mp.Medium(epsilon=2.25)  # n=1.5
# For anisotropic, use mp.Medium(epsilon_diag=[eps_x, eps_y, eps_z])

# Geometry: Layered stack
geometry = [mp.Block(material=glass, center=mp.Vector3(0.5), size=mp.Vector3(1, mp.inf, mp.inf))]

# Source: Gaussian pulse
f_center = 1.0 / (0.8 * cell_size.x)  # Central frequency
sources = [mp.Source(mp.GaussianSource(frequency=f_center, fwidth=f_center/10),
                     center=mp.Vector3(-0.5), component=mp.Ey)]

# Monitors for reflection/transmission
refl_flux = mp.FluxRegion(center=mp.Vector3(-0.5), size=mp.Vector3(0, mp.inf, mp.inf), component=mp.Ey)
trans_flux = mp.FluxRegion(center=mp.Vector3(0.5), size=mp.Vector3(0, mp.inf, mp.inf), component=mp.Ey)

sim = mp.Simulation(cell_size=cell_size, resolution=resolution, geometry=geometry,
                    sources=sources, boundary_layers=pml_layers,
                    flux_regions=[refl_flux, trans_flux])

sim.run(until=100)  # Run simulation

# Compute R and T
R = sim.get_flux(refl_flux) / mp.source_power(sources[0])
T = sim.get_flux(trans_flux) / mp.source_power(sources[0])
A = 1 - R - T  # Absorption (0 for non-lossy materials)

print(f"Reflection: {R}, Transmission: {T}, Absorption: {A}")

ImportError: cannot import name 'Vector3' from 'meep' (C:\Users\esteb\miniconda3\envs\tmm\lib\site-packages\meep\__init__.py)

In [ ]:
import tidy3d as td
import numpy as np

# Simulation setup
freq0 = 1e14  # Central frequency (Hz)
fwidth = 0.1 * freq0  # Bandwidth
sim_size = [10, 10, 5]  # um

# Materials
air = td.Medium(permittivity=1.0)
glass = td.Medium(permittivity=2.25)  # Isotropic n=1.5
# For anisotropic: glass = td.Medium(epsilon_diag=[2.25, 2.25, 4.0])

# Geometry: Layer
layer = td.Structure(geometry=td.Box(size=(np.inf, np.inf, 1)), medium=glass)

# Source: Plane wave
source = td.PlaneWaveSource(source_time=td.GaussianPulse(freq0, fwidth), direction='+',
                            pol_angle=0, size=(np.inf, np.inf, 0), center=(0, 0, -1))

# Monitors for R/T
refl_mon = td.FluxMonitor(center=(0, 0, -1.5), size=(10, 10, 0), freqs=[freq0])
trans_mon = td.FluxMonitor(center=(0, 0, 1.5), size=(10, 10, 0), freqs=[freq0])

# Simulation
sim = td.Simulation(size=sim_size, grid_spec=td.GridSpec.auto(min_steps_per_wvl=20),
                    structures=[layer], sources=[source], monitors=[refl_mon, trans_mon])

# Run (cloud-based)
task = web.run(sim, task_name="layer_stack", path_dir="data")

# Post-process
R = np.real(task.get_intensity(refl_mon))
T = np.real(task.get_intensity(trans_mon))
A = 1 - R - T
print(f"Reflection: {R}, Transmission: {T}, Absorption: {A}")